In [1]:
import json
import os
import platform
import sys
from datetime import datetime

import pytz

import src.data.Dataset as dt

# ----------------------------------------------------------------------------------------------------------------------
# Umbral de confianza para la detección de objetos
threshold_initial = float(0.8)

# ----------------------------------------------------------------------------------------------------------------------
# Configuración de las rutas de los archivos

# Ruta de las imagenes a procesar
dataset_files = os.path.abspath("resources/input/emptyNonEmptyDataset")

# Ruta del fichero CSV con las anotaciones
csv_file = os.path.abspath("data/processed/10000Train.csv")


# ----------------------------------------------------------------------------------------------------------------------
# Datos para generar las rutas de salida

# Ruta de partida para los ficheros de salida
output_path = os.path.abspath("resources/output_json/")

# ----------------------------------------------------------------------------------------------------------------------
# Fichero del modelo de deteccion
detection_model = os.path.abspath("models/MegaDetector v4.1, 2020.04.27/md_v4.1.0.pb")

# ----------------------------------------------------------------------------------------------------------------------
# Ruta de repositorios necesarios
ai4eutils_path = os.path.abspath("data/external/ai4eutils")
CameraTraps_path = os.path.abspath("data/external/CameraTraps")

try:
    os.environ["PYTHONPATH"]
except KeyError:
    os.environ["PYTHONPATH"] = ""
if platform.system() == "Windows":
    os.environ["PYTHONPATH"] += ";" + ai4eutils_path
    os.environ["PYTHONPATH"] += ";" + CameraTraps_path
else:
    os.environ["PYTHONPATH"] += ":" + ai4eutils_path
    os.environ["PYTHONPATH"] += ":" + CameraTraps_path

In [2]:
# !python ./data/external/CameraTraps/detection/run_tf_detector_batch.py "$detection_model" "$dataset_files" "$file_path" --recursive --threshold "$threshold" 

In [3]:
coverage = False
threshold = threshold_initial

while not coverage:
    threshold_str = str(threshold).replace(".", "-")

    date_time = datetime.now(pytz.timezone("Europe/Madrid")).strftime("%Y-%m-%d_%H-%M")
    name = date_time + "_" + threshold_str

    folder_name = os.path.join(output_path, name)
    os.makedirs(folder_name, exist_ok=True)

    file_path = folder_name + os.sep + name + ".json"

    checkpoint_path = os.path.join(folder_name, "checkpoints")

    checkpoint_frequency = 5

    command = f'python data/external/CameraTraps/detection/run_tf_detector_batch.py "{detection_model}" "{dataset_files}" "{file_path}" --recursive --threshold "{threshold}" --checkpoint_path "{checkpoint_path}" --checkpoint_frequency "{checkpoint_frequency}"'
    os.system(command)

    with open(file_path, "r") as file:
        data = json.load(file)

    dataset = dt.load_from_csv(csv_file)
    dataset = dt.convert_csv_to_abstract(dataset, dataset_files)

    detections_label = []
    positives_detected = []
    false_positives = []
    
    for file_name, label in zip(dataset["file_name"].values, dataset["label"].values):
        matching_images = [
            image for image in data["images"] if image["file"] == file_name
        ]

        image = matching_images[0]

        detections = image["detections"]
        detection_label = 0
        if len(detections) > 0:
            detection_label = 1
        detections_label.append(detection_label)

        false_positive = 0
        if label == 1 and detection_label == 1:
            positive_detected = 1
        elif label == 1 and detection_label == 0:
            positive_detected = 0
        elif label == 0:
            positive_detected = -1
            if detection_label == 1:
                false_positive = 1
        positives_detected.append(positive_detected)
        false_positives.append(false_positive)

    dataset["detection_label"] = detections_label
    dataset["positive_detected"] = positives_detected
    dataset["is_false_positive"] = false_positives

    path = folder_name + os.sep + name + ".csv"
    dt.dataset_to_csv(dataset, path)

    if 0 in positives_detected:
        threshold = float(round((threshold - 0.1),1))
    else:
        coverage = True

print()
print("CSV file generated: " + path)
print("Threshold: " + str(threshold))

TensorFlow version: 2.10.0
Metal device set to: Apple M3

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Is GPU available? tf.test.is_gpu_available: True
TensorFlow version: 2.10.0
tf.test.is_gpu_available: True
12 image files found in the input directory
The checkpoint file will be written to /Users/carlos/Library/Mobile Documents/com~apple~CloudDocs/WORKSPACE/TFG-Cameratraps_Detection/resources/output_json/2024-05-23_23-55_0-7/checkpoints
TFDetector: Loading graph...


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
2024-05-23 23:55:03.708353: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-05-23 23:55:03.708441: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-05-23 23:55:03.710298: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-05-23 23:55:03.710305: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (devi

TFDetector: Detection graph loaded.
Loaded model in 3.37 seconds
Processing image /Users/carlos/Library/Mobile Documents/com~apple~CloudDocs/WORKSPACE/TFG-Cameratraps_Detection/resources/input/prueba/c4728IM000143.JPG


2024-05-23 23:55:07.079365: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-05-23 23:55:07.079386: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
  0%|          | 0/12 [00:00<?, ?it/s]2024-05-23 23:55:07.230791: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2024-05-23 23:55:07.327788: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-05-23 23:55:07.356470: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-05-23 23:55:08.638633: W tensorflow

Processing image /Users/carlos/Library/Mobile Documents/com~apple~CloudDocs/WORKSPACE/TFG-Cameratraps_Detection/resources/input/prueba/lince144 (245).JPG


...

Processing image /Users/carlos/Library/Mobile Documents/com~apple~CloudDocs/WORKSPACE/TFG-Cameratraps_Detection/resources/input/prueba/37_20210319 (943).JPG
Writing a new checkpoint after having processed 10 images since last restart
Processing image /Users/carlos/Library/Mobile Documents/com~apple~CloudDocs/WORKSPACE/TFG-Cameratraps_Detection/resources/input/prueba/20_20201024 (409).JPG


 92%|█████████▏| 11/12 [00:42<00:03,  3.27s/it]

Processing image /Users/carlos/Library/Mobile Documents/com~apple~CloudDocs/WORKSPACE/TFG-Cameratraps_Detection/resources/input/prueba/586ae177-23d2-11e8-a6a3-ec086b02610b.jpg


100%|██████████| 12/12 [00:47<00:00,  3.97s/it]


Finished inference in 51.82 seconds
Output file saved at /Users/carlos/Library/Mobile Documents/com~apple~CloudDocs/WORKSPACE/TFG-Cameratraps_Detection/resources/output_json/2024-05-23_23-55_0-7/2024-05-23_23-55_0-7.json
Deleted checkpoint file /Users/carlos/Library/Mobile Documents/com~apple~CloudDocs/WORKSPACE/TFG-Cameratraps_Detection/resources/output_json/2024-05-23_23-55_0-7/checkpoints
Done!
El archivo /Users/carlos/Library/Mobile Documents/com~apple~CloudDocs/WORKSPACE/TFG-Cameratraps_Detection/data/processed/prueba.csv se ha abierto con éxito.
El conjunto de datos se ha guardado correctamente en /Users/carlos/Library/Mobile Documents/com~apple~CloudDocs/WORKSPACE/TFG-Cameratraps_Detection/resources/output_json/2024-05-23_23-55_0-7/2024-05-23_23-55_0-7.csv.

CSV file generated: /Users/carlos/Library/Mobile Documents/com~apple~CloudDocs/WORKSPACE/TFG-Cameratraps_Detection/resources/output_json/2024-05-23_23-55_0-7/2024-05-23_23-55_0-7.csv
Threshold: 0.7
